In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.common.by import By
import re


In [44]:
url = "https://www.sanfoundry.com/docker-mcq-multiple-choice-questions/"


In [45]:
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
# options.add_argument("--headless") # dung de kh mo Chrome


In [46]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get(url)
time.sleep(3)  # để page load + tránh anti-bot

#click xem dap an
buttons = driver.find_elements(By.XPATH, "//a[contains(text(),'View Answer')]")
for btn in buttons:
    driver.execute_script("arguments[0].click();", btn)
    time.sleep(0.2)
    
html = driver.page_source

driver.quit()


In [47]:
soup = BeautifulSoup(html, "html.parser")

mcqs = []

for p in soup.find_all("p"):
    text = p.get_text("\n", strip=True)

    if not re.match(r"^\d+\.", text):
        continue

    lines = text.split("\n")

    question = lines[0]
    options = [line for line in lines[1:] if line.startswith(("a)", "b)", "c)", "d)"))]

    answer_div = p.find_next_sibling("div", class_="collapseomatic_content")

    answer = ""
    explanation = ""

    if answer_div:
        for line in answer_div.get_text("\n", strip=True).split("\n"):
            if line.startswith("Answer:"):
                answer = line.replace("Answer:", "").strip()
            elif line.startswith("Explanation:"):
                explanation = line.replace("Explanation:", "").strip()

    mcqs.append([question, *options, answer, explanation])

In [48]:
file_name = "data_crawler/docker_mcq_sanfoundry.csv"

with open(file_name, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Question",
        "Option A", "Option B", "Option C", "Option D",
        "Correct Answer",
        "Explanation"
    ])
    writer.writerows(mcqs)

print(f"Đã lưu {len(mcqs)} câu hỏi vào file: {file_name}")



Đã lưu 46 câu hỏi vào file: data_crawler/docker_mcq_sanfoundry.csv


In [52]:
import csv
from docx import Document

CSV_FILE = "data_crawler/docker_mcq_sanfoundry.csv"
DOC_FILE = "docker_mcq.docx"

doc = Document()
doc.add_heading("Docker Multiple Choice Questions", level=1)

with open(CSV_FILE, encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        doc.add_paragraph(row["Question"])

        options = {
            "a": row["Option A"],
            "b": row["Option B"],
            "c": row["Option C"],
            "d": row["Option D"],
        }

        correct = row["Correct Answer"].strip().lower()

        for key, text in options.items():
            p = doc.add_paragraph()
            run = p.add_run(text)

            if key == correct:
                run.bold = True

        doc.add_paragraph("")  # dòng trống giữa các câu

doc.save(DOC_FILE)
print("✅ Đã tạo file docker_mcq.docx (không explanation, không lặp)")


✅ Đã tạo file docker_mcq.docx (không explanation, không lặp)
